In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from pathlib import Path
# from tqdm.notebook import tqdm
from imageio import imread


In [19]:
def read_grayscale_pngs(path, width=20, height=13):
    path = Path(path)
    # print(len([name for name in os.listdir('{}/.'.format(path)) if os.path.isfile(name)]))
    num_files = len(list(path.glob('*'))) # Calculate amount of files in directory
    # num_files = len([f for f in path.iterdir() if path.joinpath(f).is_file()]) # Calculate amount of files in directory

    ids = np.empty(num_files)
    images = np.empty((num_files, 13, 20))

    for i, image_path in enumerate(path.glob('*.png')):
        images[i] = np.array(imread(image_path))[:, :, 0] # Pixel data: It's grayscale so take only Red values from [R, G, B, A]
    return images

In [21]:
# legal = np.concatenate(
#     (
#         read_grayscale_pngs("cleaned/legal"),
#         read_grayscale_pngs("shifted/legal"),
#         read_grayscale_pngs("mirrored/legal"),
#     ),
#     axis=1
# )
illegal = np.concatenate(
    (
        read_grayscale_pngs("cleaned/illegal"),
        read_grayscale_pngs("shifted/illegal"),
        read_grayscale_pngs("mirrored/illegal")
    ),
    axis=0
)


ValueError: Could not find a format to read the specified file in single-image mode

In [ ]:


model = keras.Sequential()
model.add(layers.Flatten())
model.add(100, activation="relu")
model.add(20, activation="relu")
model.add(1, activation="sigmoid")

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
